In [52]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K


In [53]:
train_set = pd.read_csv("../input/train.csv")
test_set = pd.read_csv("../input/test.csv")

In [54]:
from sklearn.preprocessing import StandardScaler
y_train = train_set.label
train_set = train_set.drop('label', axis=1)
scaler = StandardScaler()
scaler.fit(train_set)
train_scaled = scaler.transform(train_set)
test_scaled = scaler.transform(test_set)

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  import sys


In [55]:
batch_size = 128
num_classes = 10
epochs = 30

In [56]:
# input image dimensions
img_rows, img_cols = 28, 28

In [57]:
if K.image_data_format() == 'channels_first':
    train_scaled = train_scaled.reshape(train_scaled.shape[0], 1, img_rows, img_cols)
    test_scaled = test_scaled.reshape(test_scaled.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    train_scaled = train_scaled.reshape(train_scaled.shape[0], img_rows, img_cols, 1)
    test_scaled = test_scaled.reshape(test_scaled.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [58]:
train_scaled = train_scaled.astype('float32')
test_scaled = test_scaled.astype('float32')
train_scaled /= 255
test_scaled /= 255

In [59]:
y_train = keras.utils.to_categorical(y_train, num_classes)

In [60]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
activation='relu',
input_shape=input_shape))
#model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(filters=32,kernel_size=5,activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [61]:
model.compile(loss=keras.losses.categorical_crossentropy,
optimizer=keras.optimizers.Adadelta(),
metrics=['accuracy'])

In [62]:
model.fit(train_scaled, y_train,
batch_size=batch_size,
epochs=epochs,
verbose=1
)

Epoch 1/30
42000/42000 [==============================] - 6s 133us/step - loss: 1.3390 - acc: 0.5319
Epoch 2/30
42000/42000 [==============================] - 5s 119us/step - loss: 0.2853 - acc: 0.9137
Epoch 3/30
42000/42000 [==============================] - 5s 119us/step - loss: 0.1766 - acc: 0.9476
Epoch 4/30
42000/42000 [==============================] - 5s 119us/step - loss: 0.1333 - acc: 0.9609
Epoch 5/30
42000/42000 [==============================] - 5s 118us/step - loss: 0.1109 - acc: 0.9670
Epoch 6/30
42000/42000 [==============================] - 5s 118us/step - loss: 0.1005 - acc: 0.9700
Epoch 7/30
42000/42000 [==============================] - 5s 118us/step - loss: 0.0867 - acc: 0.9741
Epoch 8/30
42000/42000 [==============================] - 5s 117us/step - loss: 0.0800 - acc: 0.9749
Epoch 9/30
42000/42000 [==============================] - 5s 118us/step - loss: 0.0743 - acc: 0.9775
Epoch 10/30
42000/42000 [==============================] - 5s 117us/step - loss: 0.0692 - a

In [63]:
results = model.predict(test_scaled)

In [64]:
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")



In [65]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("MNIST-CNN.csv",index=False)
